# One run full walkthrough
* Do the full walkthorugh on the large data set
* Refactor the source code and bring it to the individual scripts
* Ensure a full run with one click

## 1 Update all data

In [1]:
# %load ../src/data/get_data.py

import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    ''' Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure
    '''
    git_pull = subprocess.Popen( "/usr/bin/git pull" ,
                         cwd = os.path.dirname( '../data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


def get_current_data_germany():
    ''' Get current data from germany, attention API endpoint not too stable
        Result data frame is stored as pd.DataFrame

    '''
    # 16 states
    #data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    # 400 regions / Landkreise
    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object=json.loads(data.content)
    full_list=[]
    for pos,each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])

    pd_full_list=pd.DataFrame(full_list)
    pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))

if __name__ == '__main__':
    get_johns_hopkins()
    #get_current_data_germany()


Error : b'From https://github.com/CSSEGISandData/COVID-19\n   67b87f29..29b85c49  master     -> origin/master\n   2c3745fe..0fa72232  web-data   -> origin/web-data\n'
out : b'Updating 67b87f29..29b85c49\nFast-forward\n csse_covid_19_data/README.md                       |    3 +\n .../csse_covid_19_daily_reports/09-10-2020.csv     | 3955 ++++++++++++\n .../csse_covid_19_daily_reports/09-11-2020.csv     | 3955 ++++++++++++\n .../csse_covid_19_daily_reports/09-12-2020.csv     | 3955 ++++++++++++\n .../csse_covid_19_daily_reports/09-13-2020.csv     | 3955 ++++++++++++\n .../csse_covid_19_daily_reports_us/09-10-2020.csv  |   59 +\n .../csse_covid_19_daily_reports_us/09-11-2020.csv  |   59 +\n .../csse_covid_19_daily_reports_us/09-12-2020.csv  |   59 +\n .../csse_covid_19_daily_reports_us/09-13-2020.csv  |   59 +\n .../time_series_covid19_confirmed_US.csv           | 6682 ++++++++++----------\n .../time_series_covid19_confirmed_global.csv       |  534 +-\n .../time_series_covid19_deaths_US.c

## 2 Process pipline

In [2]:
# %load ../src/data/process_JH_data.py
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set

    '''

    data_path = '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw = pd.read_csv(data_path)

    pd_data_base = pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state'] = pd_data_base['state'].fillna('no')

    pd_data_base = pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model = pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date'] = pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: ' + str(pd_relational_model.shape[0]))

if __name__ == '__main__':

    store_relational_JH_data()


 Number of rows stored: 62776


## 3 Filter and Doubling Rate & SIR Model calculation

In [3]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal
from scipy import optimize
from scipy import integrate


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


In [4]:

def SIR_model_t(SIR, t, beta, gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta:

        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)

    '''

    S, I, R = SIR
    dS_dt = -beta * S * I / N0  # S*I is the
    dI_dt = beta * S * I / N0 - gamma * I
    dR_dt = gamma * I
    return dS_dt, dI_dt, dR_dt

def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

def SIR(data):
    popt = [0.4, 0.1]
    ydata = data
    t = np.arange(len(ydata))
    try:
        popt, pcov = optimize.curve_fit(fit_odeint, t, ydata, maxfev=5000)
        perr = np.sqrt(np.diag(pcov))
        
        # get the final fitted curve
        return fit_odeint(t, *popt)
    
    except ValueError:
        print('RunTimeError')
        return [0] * len(ydata)
            

In [5]:
pd_JH_data = pd.read_csv('../data/processed/COVID_relational_confirmed.csv', sep=';', parse_dates=[0])
pd_JH_data = pd_JH_data.sort_values('date', ascending=True).copy()
pd_JH_data.drop('state', axis=1, inplace=True)

# Create new Frame
newFrame = pd.DataFrame()
for col in pd_JH_data.columns:
    newFrame[col] = ''    
    
#if testFrame.reindex(sorted(testFrame.columns), axis=1).columns.all() == newFrame.reindex(sorted(newFrame.columns), axis=1).columns.all():        
    
for date in pd.unique(pd_JH_data['date']):
    for country in pd.unique(pd_JH_data['country']):
        singleFrame = pd_JH_data[(pd_JH_data['date'] == date) & (pd_JH_data['country'] ==  country)]
        singleFrame            = pd.DataFrame(singleFrame.sum()).T
        singleFrame['date']    = date
        singleFrame['country'] = country
        
        newFrame = newFrame.append(singleFrame)

newFrame = newFrame.reset_index()        
newFrame.to_csv('../data/processed/COVID_relational_confirmed_SIR.csv',sep=';',index=False)

pd_JH_data = newFrame.copy()
        

    

In [6]:
pd_JH_data.head()

,index,date,country,confirmed
0,0,2020-01-22,Canada,0
1,0,2020-01-22,"Korea, South",1
2,0,2020-01-22,Kosovo,0
3,0,2020-01-22,Kuwait,0
4,0,2020-01-22,Kyrgyzstan,0


In [7]:
I0 = 27
S0 = pow(10, 6)
R0 = 0

N0 = pow(10, 6)
beta = 0.4
gamma = 0.1

pd_result_SIR = pd_JH_data.copy()
pd_result_SIR['SIR_static'] = 0


for each in pd.unique(pd_result_SIR['country']):
    
    #print(each)
    
    # Extract y data
    ydata_SIR = np.array(pd_result_SIR[pd_result_SIR['country'] == each]['confirmed'][35:])            
    t = np.arange(len(ydata_SIR))        

    # Calc SIR Data
    fitted_SIR = SIR(ydata_SIR.copy())    

    # Create a new dataframe with SIR data for update function
    SIRFrame = pd.DataFrame({'date': pd_result_SIR[pd_result_SIR['country'] == each]['date'][35:]})
    SIRFrame['SIR_static'] = fitted_SIR
    SIRFrame['country'] = each

    pd_result_SIR.update(SIRFrame['SIR_static'])        
    

/home/nick/anaconda3/lib/python3.8/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-4-b500a10bf243>:17: RuntimeWarning: overflow encountered in double_scalars
  dI_dt = beta * S * I / N0 - gamma * I
<ipython-input-4-b500a10bf243>:18: RuntimeWarning: overflow encountered in double_scalars
  dR_dt = gamma * I
<ipython-input-4-b500a10bf243>:16: RuntimeWarning: overflow encountered in double_scalars
  dS_dt = -beta * S * I / N0  # S*I is the


In [8]:
pd_result_SIR.head()

,index,date,country,confirmed,SIR_static
0,0,2020-01-22,Canada,0,0.0
1,0,2020-01-22,"Korea, South",1,0.0
2,0,2020-01-22,Kosovo,0,0.0
3,0,2020-01-22,Kuwait,0,0.0
4,0,2020-01-22,Kyrgyzstan,0,0.0


In [9]:
pd_JH_data = pd.read_csv('../data/processed/COVID_relational_confirmed.csv', sep=';', parse_dates=[0])
pd_JH_data = pd_JH_data.sort_values('date', ascending=True).copy()

In [10]:
pd_JH_data = pd_result_SIR.copy()
pd_JH_data = pd_JH_data.reset_index()
#pd_JH_data.drop('index', axis=1, inplace=True)
#pd_JH_data.rename(columns={'level_0':'index'})
pd_JH_data['state'] = ''
pd_JH_data


,level_0,index,date,country,confirmed,SIR_static,state
0,0,0,2020-01-22,Canada,0,0.000000,
1,1,0,2020-01-22,"Korea, South",1,0.000000,
2,2,0,2020-01-22,Kosovo,0,0.000000,
3,3,0,2020-01-22,Kuwait,0,0.000000,
4,4,0,2020-01-22,Kyrgyzstan,0,0.000000,
...,...,...,...,...,...,...,...
44363,44363,0,2020-09-13,Argentina,555537,989.831321,
44364,44364,0,2020-09-13,Antigua and Barbuda,95,98.130972,
44365,44365,0,2020-09-13,Angola,3388,3477.475411,
44366,44366,0,2020-09-13,Andorra,1344,784.914842,


In [11]:
pd_result_larg = calc_filtered_data(pd_JH_data)
pd_result_larg = calc_doubling_rate(pd_result_larg)
pd_result_larg = calc_doubling_rate(pd_result_larg, 'confirmed_filtered')

mask = pd_result_larg['confirmed'] > 100
pd_result_larg['confirmed_filtered_DR'] = pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv', sep=';', index=False)

In [12]:
#pd_result_larg[pd_result_larg['country'] == 'US'][30:].head(50)

## 4 Visual Board

In [ ]:
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())
df_input_large = pd.read_csv('../data/processed/COVID_final_set.csv',sep=';')


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label':each, 'value':each} for each in df_input_large['country'].unique()],
        value=['US', 'Germany', 'Italy'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
        {'label': 'Timeline SIR Model applied', 'value': 'SIR_static'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list, show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date', 'SIR_static']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date', 'SIR_static']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }
def visualize():
    app.run_server(debug=True, use_reloader=False)

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)


/home/nick/Documents/Data Science/OwnProject/ads_covid-19/notebooks
Running on http://127.0.0.1:8050/
Debugger PIN: 956-538-988
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
